# Generic CNN for MNIST
The goal of this notebook ist to generalize the necessary building blocks for a CNN for our MNIST. This allows fast experimentation with different network designs. We also seek to integrate Tensorboard support for this. 

  Ideally it allows us to phase out all complexity related to the general purpose apis so that we only need to focus on the abstract data that actually differs between different network designs.

## First Approach
We base our first approach on the tutorial for TensorBoard with MNIST.

The original code comes from https://raw.githubusercontent.com/tensorflow/tensorflow/r0.11/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py.

We use it as a base to iterate on.

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
from functools import *
from datetime import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

class dotdict(dict):
    __getattr__ = dict.get


logdir = lambda d: "/".join([d, datetime.now().strftime("%Y%m%d-%H%M%S")])
    
FLAGS = dotdict({'fake_data':False,
                 'max_steps':1000,
                 'learning_rate':0.001,
                 'dropout':0.9,
                 'data_dir':'/tmp/data',
                 'summaries_dir':'/tmp/mnist_logs',
                 'epochs' : 1 })

n_inputs = 784
n_classes = 10
batch_size = 50

#setup placeholders
x = tf.placeholder('float',[None,n_inputs])
y = tf.placeholder('float',[None,n_classes])
layer_shape = [] #placeholder

def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.scalar_summary('mean/' + name, mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.scalar_summary('stddev/' + name, stddev)
      tf.scalar_summary('max/' + name, tf.reduce_max(var))
      tf.scalar_summary('min/' + name, tf.reduce_min(var))
      tf.histogram_summary(name, var)

def weight(w,name,**kwargs):
    print(name)
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(w, stddev=0.1)
    ret =  tf.Variable(initial)
    variable_summaries(ret,name+'/weights')
    return ret
    
def bias(b,name,**kwargs):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=b)
    ret = tf.Variable(initial)
    variable_summaries(ret,name+'/bias')
    return ret
        
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def conv2d(x, w, b, k=1):
    print({'x':x,'w':w,'b':b})
    x = tf.nn.conv2d(x, w, strides=[1, k, k, 1] , padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def conv(x,w,b,name,k_c=1,k_p=2,**kwargs):
    # Convolution
    conv = conv2d(x,w, b,k=k_c)
    # Pooling
    pool = maxpool2d(conv, k=k_p)
    return pool

def fc(x,w,b,name,k_c=1,k_p=2,**kwargs):
    pre= tf.add(tf.matmul(x, w), b)
    tf.histogram_summary(name+'/pre_activations',pre)
    return pre

def act(pre,name,**kwargs):
    ret = tf.nn.relu(pre)
    tf.histogram_summary(name+'/activations',ret)
    return ret

layer_type = ({'conv':conv, 'fc':fc}).get

def conv_neural_net(x,lshp = layer_shape, lt=layer_type):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    def process_layer(**args):
        print(args)
        wl,bl=(i(**args) for i in (weight,bias))
        return lt(args['fn'])(**{**args,'w':wl,'b':bl})
    def fun(acc,e):
        prev, cur= e
        print(cur['name'])
        if (prev['fn'] == 'conv' and cur['fn']!='conv'):
            acc = tf.reshape(acc,[-1,cur['w'][0]])
        if (prev['fn'] == 'fc' and cur['fn']== 'fc'):
            acc = act(acc,prev['name'])
        if (prev['fn']!='conv' and cur['fn']=='conv'):
            acc = tf.reshape(acc, shape=[-1, 28, 28, 1])
        return process_layer(**{**cur, 'x':acc})
    mixed = zip([{'fn':None},*lshp],lshp)
    return reduce(fun,mixed,x)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#list of stacked layers
#name is name for logging, w is weight shape, b is bias shape,
# fn is name for layer type (or rather layer construction function)
layer_shape = [
    {'name':'c1','w':[5,5,1,32],'b':[32],'fn':'conv'},
    {'name':'c2', 'w':[5,5,32,64], 'b':[64],'fn':'conv'},
    {'name':'d1','w':[7*7*64,1024], 'b':[1024],'fn':'fc'},
    {'name':'out','w':[1024,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [3]:
def cost(pred,y):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
    tf.scalar_summary('cross entropy',loss)
    return loss
    
def opt(loss,learning_rate):
    return tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
def acc(pred,y):
    correct = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    tf.scalar_summary('accuracy',accuracy)
    return accuracy

In [4]:
def train_and_test_conv_neural_net(x, y,pred, summaries_dir,learning_rate,epochs = 1,**kwargs):
    loss = cost(pred,y)
    optim = opt(loss,learning_rate)
    accuracy = acc(pred,y)
    merged = tf.merge_all_summaries()
    with tf.Session() as sess:
        summaries_dir = logdir(summaries_dir)
        train_writer = tf.train.SummaryWriter(summaries_dir + '/train',sess.graph)
        test_writer = tf.train.SummaryWriter(summaries_dir + '/test')
        sess.run(tf.initialize_all_variables())
        idx = lambda e,i: int(mnist.train.num_examples/batch_size*e+i)
        for e in range(epochs):
            epoch_loss = 0
            for i in range(int(mnist.train.num_examples/batch_size)):
                if i % 1000 == 0:
                    summary, accu = sess.run([merged,accuracy],feed_dict = {x:mnist.test.images, y:mnist.test.labels})
                    test_writer.add_summary(summary,idx(e,i))
                else:
                    epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                    _, c,summary = sess.run([optim, loss,merged], feed_dict = {x:epoch_x, y:epoch_y})
                    train_writer.add_summary(summary, idx(e,i))
                    epoch_loss = epoch_loss + c
            print('Epoch ' + str(e+1) + ' out of ' + str(epochs) + ' / Loss: ' + str(epoch_loss))
            print('Accuracy: ' + str(accuracy.eval({x:mnist.test.images, 
                                                y:mnist.test.labels})))
        print('Training finished!')

In [5]:
pred= conv_neural_net(x,layer_shape)

c1
{'w': [5, 5, 1, 32], 'fn': 'conv', 'x': <tf.Tensor 'Reshape_1:0' shape=(?, 28, 28, 1) dtype=float32>, 'b': [32], 'name': 'c1'}
c1
{'w': <tensorflow.python.ops.variables.Variable object at 0x10b6f8668>, 'x': <tf.Tensor 'Reshape_1:0' shape=(?, 28, 28, 1) dtype=float32>, 'b': <tensorflow.python.ops.variables.Variable object at 0x10b7d35c0>}
c2
{'w': [5, 5, 32, 64], 'fn': 'conv', 'x': <tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>, 'b': [64], 'name': 'c2'}
c2
{'w': <tensorflow.python.ops.variables.Variable object at 0x1125d0780>, 'x': <tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>, 'b': <tensorflow.python.ops.variables.Variable object at 0x1044259e8>}
d1
{'w': [3136, 1024], 'fn': 'fc', 'x': <tf.Tensor 'Reshape_2:0' shape=(?, 3136) dtype=float32>, 'b': [1024], 'name': 'd1'}
d1
out
{'w': [1024, 10], 'fn': 'fc', 'x': <tf.Tensor 'Relu_2:0' shape=(?, 1024) dtype=float32>, 'b': [10], 'name': 'out'}
out


In [6]:
train_and_test_conv_neural_net(x,y,pred,**{**FLAGS, 'epochs': 3})

KeyboardInterrupt: 

## Misc Legacy Snippets
please ignore

In [1]:
# modules:
def inputs():
    pass
    
def inference():
    pass
    
def loss():
    pass

def train():
    pass

def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.scalar_summary('mean/' + name, mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.scalar_summary('stddev/' + name, stddev)
      tf.scalar_summary('max/' + name, tf.reduce_max(var))
      tf.scalar_summary('min/' + name, tf.reduce_min(var))
      tf.histogram_summary(name, var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights, layer_name + '/weights')
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases, layer_name + '/biases')
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.histogram_summary(layer_name + '/pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.histogram_summary(layer_name + '/activations', activations)
      return activations

def train():
  # Import data
  mnist = input_data.read_data_sets(FLAGS.data_dir,
                                    one_hot=True,
                                    fake_data=FLAGS.fake_data)

  sess = tf.InteractiveSession()

  # Create a multilayer model.

  # Input placeholders
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.image_summary('input', image_shaped_input, 10)

  

  hidden1 = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.scalar_summary('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the
    # raw outputs of the nn_layer above, and then average across
    # the batch.
    diff = tf.nn.softmax_cross_entropy_with_logits(y, y_)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
    tf.scalar_summary('cross entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy', accuracy)

  # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
  merged = tf.merge_all_summaries()
  train_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/train',
                                        sess.graph)
  test_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/test')
  tf.initialize_all_variables().run()

  # Train the model, and also write summaries.
  # Every 10th step, measure test-set accuracy, and write test summaries
  # All other steps, run train_step on training data, & add training summaries

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or FLAGS.fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
      k = FLAGS.dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

  for i in range(FLAGS.max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  train_writer.close()
  test_writer.close()


def main():
  if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
  tf.gfile.MakeDirs(FLAGS.summaries_dir)
  train()

In [2]:
main()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy at step 0: 0.0438
Accuracy at step 10: 0.6836
Accuracy at step 20: 0.8173
Accuracy at step 30: 0.8474
Accuracy at step 40: 0.8707
Accuracy at step 50: 0.8793
Accuracy at step 60: 0.8848
Accuracy at step 70: 0.8819
Accuracy at step 80: 0.884
Accuracy at step 90: 0.8966
Adding run metadata for 99
Accuracy at step 100: 0.9085
Accuracy at step 110: 0.9136
Accuracy at step 120: 0.9173
Accuracy at step 130: 0.9179
Accuracy at step 140: 0.9208
Accuracy at step 150: 0.9164
Accuracy at step 160: 0.9279
Accuracy at step 170: 0.9243
Accuracy at step 180: 0.9224
Accuracy at step 190: 0.9219
Adding run metadata for 199
Accuracy at step 200: 0.9274
Accuracy at step 210: 0.9304
Accuracy at step 220: 0.9347
Accuracy at step 230: 0.9353
Accuracy at step 240: 0.9352
Accuracy at step 250: 0.9307
Accuracy at s